# Задание 1

Имплементируйте алгоритм Леска (описание есть в семинаре) и оцените качество его работы на датасете `data/corpus_wsd_50k.txt`

В качестве метрики близости вы должны попробовать два подхода:

1) Jaccard score на множествах слов (определений и контекста)
2) Cosine distance на эмбедингах sentence_transformers

В качестве метрики используйте accuracy (% правильных ответов). Предсказывайте только многозначные слова в датасете

Контекст вы можете определить самостоятельно (окно вокруг целевого слова или все предложение). Также можете поэкспериментировать с предобработкой для обоих методов.

In [27]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dariachelnokova/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [28]:
from nltk.corpus import wordnet as wn

In [29]:
corpus_wsd = []
corpus = open('corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

In [9]:
#!pip install nltk


## 1.1. Jaccard score на множествах слов (определений и контекста)

In [30]:
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
true = 0  # Количество правильных ответов
total = 0  # Общее количество слов
window_size = 4
for sent in corpus_wsd[0:2000]:
    for word_index, word_info in enumerate(sent):
        # Извлекаем информацию о значении, лемме и слове по индексам
        meaning = word_info[0]
        lemma = word_info[1]
        word = word_info[2]
        
        # Проверяем, является ли слово многозначным (имеет значение)
        if not meaning:
            continue  # Если слово однозначное, пропускаем его и переходим к следующему
        
        total += 1  # Увеличиваем счетчик общего числа оцененных многозначных слов
        
        # Создаем окно вокруг слова
        start_index = max(0, word_index - window_size)
        end_index = min(len(sent), word_index + window_size + 1)
        context_words = [sent[i][2] for i in range(start_index, end_index) if i != word_index]
        
        # Удаляем стоп-слова из контекста
        context_words = [word for word in context_words if word.lower() not in stop_words]
        
        # Создаем контекст из обработанных слов
        context = " ".join(context_words)
        
        # Создаем множество слов из контекста
        context_set = set(context.split())
        
        best_meaning = None # Хранения наилучшего значения слова из WordNet
        best_jaccard = float('inf')  # Хранения наивысшего Jaccard Score
        
        # Проходимся по всем синсетам слова из WordNet
        for synset in wn.synsets(lemma):
            # Получаем определение и создаем множество слов из определения
            meaning_words = set(synset.definition().split())
            
            # Удаление стоп-слов из определения
            meaning_words = [word for word in meaning_words if word.lower() not in stop_words]
            
            # Вычисляем Jaccard Score между контекстом и определением
            intersection = len(context_set & set(meaning_words))
            union = len(context_set | set(meaning_words))
            jaccard_score = intersection / union
            
            # Если текущий Jaccard Score лучше (меньше) предыдущего лучшего
            if jaccard_score < best_jaccard:
                best_jaccard = jaccard_score
                best_meaning = synset
                
        # Сравниваем наилучшее значение с фактическим значением многозначного слова из WordNet
        if best_meaning == wn.lemma_from_key(meaning).synset():
            true += 1  # Если значения совпали, увеличиваем счетчик правильных ответов

# Вычисляем процент правильных ответов
accuracy_percentage = (true / total) * 100
rounded_accuracy = round(accuracy_percentage, 1)
print(f'Доля правильных ответов: {rounded_accuracy}%')


Доля правильных ответов: 54.5%


## 1.2. Cosine distance на эмбедингах sentence_transformers

In [31]:
!python3 -m pip install torch torchvision torchaudio
!python3 -m pip install sentence_transformers transformers accelerate -U

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [32]:
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sentence_transformers import SentenceTransformer

In [47]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
true = 0  # Количество правильных ответов
total = 0  # Общее количество слов
window_size = 4

def embed(text):
    return model.encode(text, convert_to_tensor=True)

for sent in corpus_wsd[0:20]:
    for word_index, word_info in enumerate(sent):
        # Извлекаем информацию о значении, лемме и слове по индексам
        meaning = word_info[0]
        lemma = word_info[1]
        word = word_info[2]
        
        # Проверяем, является ли слово многозначным (имеет значение)
        if not meaning:
            continue  # Если слово однозначное, пропускаем его и переходим к следующему
        
        total += 1  # Увеличиваем счетчик общего числа оцененных многозначных слов
        
        # Создаем окно вокруг слова
        start_index = max(0, word_index - window_size)
        end_index = min(len(sent), word_index + window_size + 1)
        context_words = [sent[i][2] for i in range(start_index, end_index) if i != word_index]
        
        # Удаляем стоп-слова из контекста
        context_words = [word for word in context_words if word.lower() not in stop_words]
        
        # Создаем контекст из обработанных слов
        context = " ".join(context_words)
        
        # Получаем значения слова из WordNet
        definitions = [synset.definition() for synset in wn.synsets(lemma)]
        
        # Вычисляем эмбеддинги для контекста и определений
        context_embedding = embed(context)
        definition_embeddings = [embed(definition) for definition in definitions]
        
        # Вычисляем Cosine Distance между контекстом и определениями
        min_distance = float('inf')
        best_meaning = None
        
        for i, definition_embedding in enumerate(definition_embeddings):
            distance = cosine_distances([context_embedding.numpy()], [definition_embedding.numpy()])[0][0] 
            if distance < min_distance:
                min_distance = distance
                best_meaning = wn.synsets(lemma)[i]
                
        # Сравниваем наилучшее значение с фактическим значением многозначного слова из WordNet
        if best_meaning == wn.lemma_from_key(meaning).synset():
            true += 1  # Если значения совпали, увеличиваем счетчик правильных ответов

# Вычисляем процент правильных ответов и выводим результат
accuracy_percentage = (true / total) * 100
rounded_accuracy = round(accuracy_percentage, 1)
print(f'Процент правильных ответов: {rounded_accuracy}%')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dariachelnokova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Процент правильных ответов: 39.2%


# Задание 2
Попробуйте разные алгоритмы кластеризации на датасете - `https://github.com/nlpub/russe-wsi-kit/blob/initial/data/main/wiki-wiki/train.csv`

Используйте код из семинара как основу. Используйте ARI как метрику качества.

Попробуйте все 4 алгоритма кластеризации, про которые говорилось на семинаре. Для каждого из алгоритмов попробуйте настраивать гиперпараметры (посмотрите их в документации). Прогоните как минимум 5 экспериментов (не обязательно успешных) с разными параметрами на каждый алгоритме кластеризации и оцените: качество кластеризации, скорость работы, интуитивность параметров.

Помимо этого также выберите 1 дополнительный алгоритм кластеризации отсюда - https://scikit-learn.org/stable/modules/clustering.html , опишите своими словами принцип его работы  и проделайте аналогичные эксперименты. 

In [3]:
#!pip install pandas

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/30/6f/910f62af8642c94acca4fff529944c1e9463cf118742f7ee1a583fc6449c/pandas-2.1.1-cp311-cp311-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/32/4d/aaf7eff5deb402fd9a24a1449a8119f00d74ae9c2efa79f8ef9994261fc2/pytz-2023.3.post1-py2.py3-none-any.whl.metadata
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
Using cached pandas-2.1.1-cp311-cp311-macosx_10_9_x86_64.whl (11.6 MB)
Using cached pytz-2023.3.post1-py2.py3-none-any.whl (502 kB)


In [19]:
import pandas as pd 

In [48]:
df = pd.read_csv('https://raw.githubusercontent.com/nlpub/russe-wsi-kit/initial/data/main/wiki-wiki/train.csv', sep='\t')

In [49]:
df.head(100)

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,1,замок,1,NaN,"0-5, 339-344",замок владимира мономаха в любече . многочисле...
1,2,замок,1,NaN,"11-16, 17-22, 188-193","шильонский замок замок шильйон ( ) , известный..."
2,3,замок,1,NaN,299-304,проведения архитектурно - археологических рабо...
3,4,замок,1,NaN,111-116,"топи с . , л . белокуров легенда о завещании м..."
4,5,замок,1,NaN,"134-139, 262-267",великий князь литовский гедимин после успешной...
...,...,...,...,...,...,...
95,96,замок,1,NaN,"163-168, 213-218, 364-369",без каминов и очагов . у входа в башню было по...
96,97,замок,1,NaN,"221-226, 276-281",v . в том сражении шотландцы потерпели сокруши...
97,98,замок,1,NaN,"0-5, 16-21, 179-184",замок данноттар замок данноттар ( ) расположен...
98,99,замок,1,NaN,232-237,. благодаря поэме байрона заключение бонивара ...


In [50]:
grouped_df = df.groupby('word')[['word', 'context', 'gold_sense_id']]

In [51]:

for key, _ in grouped_df:
    print(grouped_df.get_group(key), "\n\n")
    break

    word                                            context  gold_sense_id
383  бор  бор ( элемент ) бор — элемент тринадцатой груп...              1
384  бор  бор - углерод - кремний семейство сплавов на о...              1
385  бор  с большим выделением теплоты , образуется окси...              1
386  бор  это объясняется прежде всего тем , что у компл...              1
387  бор  совсем малочисленна . элементарный бор в приро...              1
388  бор  действующем при месторождении горно - химическ...              1
389  бор  b c ) . при нагревании в атмосфере кислорода и...              1
390  бор  собственных минералов бора в чужих минералах о...              1
391  бор  бор - углерод - кремний семейство сплавов на о...              1
392  бор  с другими галогенами с образованием тригалоген...              1
393  бор  и в стабильны и входят в состав природного бор...              1
394  бор  номером . обозначается символом b ( ) . в своб...              1
395  бор  и взаимные пере

In [81]:
from sklearn.cluster import KMeans, AffinityPropagation, DBSCAN, AgglomerativeClustering, MeanShift
import numpy as np
from sklearn.metrics import adjusted_rand_score

In [53]:
# embedding model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embed = model.encode

In [54]:
# Группируем данные по словам
grouped_df = df.groupby('word')[['word', 'context', 'gold_sense_id']]

In [55]:
# Создаем список для хранения ARI для каждого слова
ARI = []

In [56]:
for key, _ in grouped_df:
    # Вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

In [57]:
    # Создаем пустую матрицу для векторов
    X = np.zeros((len(texts), 768))

In [58]:
    # Переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

In [78]:
    # выбираем один из алгоритмов
    cluster = KMeans(n_clusters=3, init='k-means++')
#    cluster = KMeans(n_clusters=10, init='random')
#    cluster = KMeans(n_clusters=3, init=10)
#    cluster = KMeans(n_clusters=5, n_init=20)
#    cluster = KMeans(n_clusters=10, n_init=5)
    
#    cluster = AffinityPropagation(damping=0.5)
#    cluster = AffinityPropagation(damping=0.6, preference=5)
#    cluster = AffinityPropagation(damping=0.7, preference=-3, affinity ='precomputed')
#    cluster = AffinityPropagation(damping=0.8, preference=20, convergence_iter ="100")
#    cluster = AffinityPropagation(damping=0.9, preference=10)


#    cluster = DBSCAN(eps=0.5, min_samples=5, metric='euclidean', algorithm='auto', leaf_size=30)
#    cluster = DBSCAN(eps=0.2, min_samples=10, metric='manhattan', algorithm='ball_tree', leaf_size=20)
#    cluster = DBSCAN(eps=0.3, min_samples=3, metric='cosine', algorithm='kd_tree', leaf_size=40)
#    cluster = DBSCAN(eps=0.4, min_samples=8, metric='chebyshev', algorithm='auto', leaf_size=50)
#    cluster = DBSCAN(eps=0.6, min_samples=12, metric='mahalanobis', algorithm='auto', leaf_size=30)


#    cluster = AgglomerativeClustering(n_clusters=3)
#    cluster = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')
#    cluster = AgglomerativeClustering(n_clusters=6, affinity='manhattan', linkage='complete')
#    cluster = AgglomerativeClustering(n_clusters=3, affinity='cosine', linkage='average')
#    cluster = AgglomerativeClustering(n_clusters=4, affinity='euclidean', linkage='ward')

### Mean Shift

Принцип работы Mean Shift заключается в смещении "средней точки"  в пространстве данных таким образом, чтобы она двигалась к месту, где плотность точек больше,  пока не достигнет максимума плотности.  Каждая точка будет отнесена к одному из кластеров на основе того, к какой средней точке она ближе.

In [82]:

#    cluster = MeanShift(bandwidth=0.5)
#    cluster = MeanShift(bin_seeding=True, min_bin_freq=5)
#    cluster = Mean_Shift(max_iter=100, cluster_all=False)
#    cluster = MeanShift(max_iter=200, bandwidth=0.6)
#    cluster = MeanShift(bin_seeding=True, min_bin_freq=10)

In [83]:
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

-0.015141540186292346
